In [1]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from codecarbon import EmissionsTracker
from metrics import get_metrics

In [2]:
dataset = "coveo"   # coveo, diginetica, rees46, retailrocket, yoochoose
dataset_path = f"../datasets/{dataset}"

In [3]:
train_path = os.path.join(dataset_path,f"{dataset}_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,f"{dataset}_processed_view_test.tsv")

In [4]:
def load_and_process_data(path):
    data = pd.read_csv(path, sep='\t')

    user_ids = data['SessionId'].astype('category').cat.codes.values
    item_ids = data['ItemId'].astype('category').cat.codes.values
    data['UserId'] = user_ids
    data['ItemId'] = item_ids

    return data, len(np.unique(user_ids)), len(np.unique(item_ids))


def create_interaction_matrix(data, num_users, num_items):
    interactions = np.zeros((num_users, num_items))
    for row in data.itertuples():
        interactions[row.UserId, row.ItemId] = 1
    return interactions


def knn_model(X_train, y_train, X_test, k=20):
    train_data = tf.constant(X_train, dtype=tf.float32)
    test_data = tf.constant(X_test, dtype=tf.float32)
    train_labels = tf.constant(y_train, dtype=tf.int32)

    distances = tf.norm(train_data[:, tf.newaxis] - test_data[tf.newaxis, :], axis=2)
    _, indices = tf.nn.top_k(-distances, k=k)
    nearest_labels = tf.gather(train_labels, indices)
    predictions = tf.math.reduce_mode(nearest_labels, axis=1)
    
    return predictions.numpy()

In [6]:
def process_data(train_path, test_path):
    # Carga y procesa los datos de entrenamiento
    train_data, num_users, num_items = load_and_process_data(train_path)

    # Crea la matriz de interacciones para los datos de entrenamiento
    X_train = create_interaction_matrix(train_data, num_users, num_items)
    y_train = X_train.copy()  # Las etiquetas son las mismas interacciones

    # Carga y procesa los datos de prueba
    test_data, _, _ = load_and_process_data(test_path)

    # Crea la matriz de interacciones para los datos de prueba
    X_test = create_interaction_matrix(test_data, num_users, num_items)

    # Normaliza los datos
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Aplicar PCA en caso de ser necesario
    # pca = PCA(n_components=100)
    # X_train = pca.fit_transform(X_train)
    # X_test = pca.transform(X_test)

    return X_train, y_train, X_test, test_data

In [ ]:
X_train, y_train, X_test, test_data = process_data(train_path, test_path)

codecarbon_tracker = EmissionsTracker()
codecarbon_tracker.start()
predictions = knn_model(X_train, y_train, X_test)
emissions = codecarbon_tracker.stop()
print(f"CO2 emissions: {emissions} kg")

# Calcula las métricas
metrics = get_metrics(f"knn_tensorflow_{dataset}", test_data, predictions)
print(metrics)

[codecarbon WARNING @ 16:02:55] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 16:02:55] [setup] RAM Tracking...
[codecarbon INFO @ 16:02:55] [setup] GPU Tracking...
[codecarbon INFO @ 16:02:55] No GPU found.
[codecarbon INFO @ 16:02:55] [setup] CPU Tracking...
[codecarbon WARNING @ 16:02:55] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 16:02:57] We saw that you have a 13th Gen Intel(R) Core(TM) i7-1355U but we don't know it. Please contact us.
[codecarbon INFO @ 16:02:57] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i7-1355U
[codecarbon INFO @ 16:02:57] >>> Tracker's metadata:
[codecarbon INFO @ 16:02:57]   Platform system: Windows-10-10.0.22631-SP0
[codecarbon INFO @ 16:02:57]   Python version: 3.11.9
[codecarbon INFO @ 16:02:57]   CodeCarbon version: 2.4.2
[codecarbon INFO @ 16:02:57]   Available RAM : 15.692 GB
[codecarbon INFO @ 16:02:57]   CPU count: 12
[codecarbon INFO @ 16:02:57]   CP

MemoryError: Unable to allocate 13.4 GiB for an array with shape (165673, 10868) and data type float64

[codecarbon INFO @ 16:05:00] Energy consumed for RAM : 0.000196 kWh. RAM Power : 5.884534835815431 W
[codecarbon INFO @ 16:05:00] Energy consumed for all CPUs : 0.001418 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 16:05:00] 0.001614 kWh of electricity used since the beginning.
[codecarbon INFO @ 16:05:02] Energy consumed for RAM : 0.000245 kWh. RAM Power : 5.884534835815431 W
[codecarbon INFO @ 16:05:02] Energy consumed for all CPUs : 0.001768 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 16:05:02] 0.002013 kWh of electricity used since the beginning.
[codecarbon INFO @ 16:05:15] Energy consumed for RAM : 0.000221 kWh. RAM Power : 5.884534835815431 W
[codecarbon INFO @ 16:05:15] Energy consumed for all CPUs : 0.001595 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 16:05:15] 0.001816 kWh of electricity used since the beginning.
[codecarbon INFO @ 16:05:17] Energy consumed for RAM : 0.000269 kWh. RAM Power : 5.884534835815431 W
[codecarbon INFO @ 16:05:17] Energy consumed for a